<a href="https://colab.research.google.com/github/alaahendam/Sentiment-analysis-project/blob/main/sentiment_analysis_arabic_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset

Dataset of Hotel Reviews scrapped from TripAdvisor.com [source: [large-arabic-sentiment-analysis-resouces](https://github.com/hadyelsahar/large-arabic-sentiment-analysis-resouces)]

In [2]:
!wget https://github.com/hadyelsahar/large-arabic-sentiment-analysis-resouces/raw/master/datasets/HTL.csv

--2020-12-23 23:10:36--  https://github.com/hadyelsahar/large-arabic-sentiment-analysis-resouces/raw/master/datasets/HTL.csv
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hadyelsahar/large-arabic-sentiment-analysis-resouces/master/datasets/HTL.csv [following]
--2020-12-23 23:10:36--  https://raw.githubusercontent.com/hadyelsahar/large-arabic-sentiment-analysis-resouces/master/datasets/HTL.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15254112 (15M) [text/plain]
Saving to: ‘HTL.csv.1’

HTL.csv.1           100%[===================>]  14.55M  --.-KB/s    in 0.06s   

2020-12-23 23:10:37 (257 MB/

# Imports

In [3]:
import os
import re
import pickle
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
%load_ext tensorboard

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras import layers
from keras.backend import clear_session

In [4]:
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [5]:
seed = 42

# Preprocessing

In [6]:
def clean_text(text):  
    # ref: https://github.com/bakrianoo/aravec
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى",
              "\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا",
               "","","","ي","",' ', ' ',' ',' ? ',' ؟ ', ' ! ']
    
    tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(tashkeel,"", text)
    
    longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(longation, subst, text)
    
    text = re.sub(r"[^\w\s]", '', text)
    text = re.sub(r"[a-zA-Z]", '', text)
    text = re.sub(r"\d+", ' ', text)
    text = re.sub(r"\n+", ' ', text)
    text = re.sub(r"\t+", ' ', text)
    text = re.sub(r"\r+", ' ', text)
    text = re.sub(r"\s+", ' ', text)
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    text = text.strip()
    
    return text

# Create DataFrame

In [7]:
data = pd.read_csv('HTL.csv')
data.head()

,text,polarity
0,"المكان الذي يمكنك فيه مراجعة الذات والتفكر هو كوكروبيت، غانا.... \r\nثمة الكثير عند زيارة غانا. وعلى الرغم من الفقر الذي سوف تلاحظه على طريقك إلى بيغ ميلي باكيارد، وجدت أن الناس في غانا يملكون ثراء القلب حتى رغم العوز. بيغ ميلي باكيارد هو مكان يمكنني فيه مراجعة الذات والتفكر . التقيت أشخاصاً كانوا يعملون على مبادرات مختلفة. بيغ ميلي هو مكان يمكننا فيه تبادل الملاحظات والآراء. بيغ ميلي مليء بالسلام وهادئ للغاية . الطعام ممتاز جداً! الطهاة ممتازون! وبالإضافة إلى ذلك, لا بد من الإشارة إلى أن العمال الذين لا يكلون، ويحرصون على سعادة نزلائهم هم من يجعلون المكان مضيافاً. هناك خيبة أمل واحدة كبيرة!! رغم أن الفنون والأشغال اليدوية يمكن شراؤها في الموقع إلا أن الأسعار غالية!!! مؤخراً أزالت السلطات في كوكروبيت محلات التجار الواقعة خارج شاطئ بيغ ميلي. كانت منتجات التجار أرخص! لا يزال هناك محل تجاري ""متجر مايسترو للفن الإفريقي في كوكروبيت"" يعرض الفنون والأشغال اليدوية الفريدة والملابس واللوحات. اشتريت لوحات مذهلة لايمكنك أن تجدها في بيغ ميلي. عندما تزور بيغ ميلي من فضلك خذ لحظة للذهاب إلى الشاطئ وعلى يمينك ستجد ""متجر مايسترو للفن الإفريقي"" المذهل. لا تنخدع بأسعار التجار في بيغ ميلي باكيارد. استمتع بالموسيقى والناس! الرجاء أن تقضي وقتاً ممتعاً.",1
1,موقع رائع وحديقة رائعة ويستحق نجمةّ إضافية \r\nعلى الرغم من أن الغرف ليست فاخرة، فهي نظيفة جداً وتعمل بشكل مثالي. بالتأكيد هناك بعض التصرفات الفردية والجماعية، ولكن كان هناك تكييف هواء كبير وجيد ونوافذ مزدوجة جيدة مع شاشات. منطقة الحديقة تحتوى على 3 مطاعم بالإضافة إلى بار رياضي. تبلغ تكلفة الإنترنت اللاسلكي 1:50 دولار للساعة، ولا يوجد خصم على الفترات الأطول، لذلك اضطررت إلى إغلاق حاسوبي أكثر. إف و بي رخيص جداً. يبدو أن هناك العديد من الزوار الأجانب هناك. 85 دولار أمريكي لليلة بما في ذلك بوفيه إفطار بمستوى نجمة واحدة. إقامة أفضل بكثير مما اعتقدت، سنعود بالتأكيد. شملت الإقامات السابقة فندق الأفريقية ريجنت و الدخول بالمطار غرب فندق (خيارريجينت أفريكان وفندق إيربيورت ويست (خيار قوي لتلك المنطقة من المدينة). حركة المرور في أكرا مُصابة بالشلل، لذا قم بعقد اجتماعاتك في فندق بالوما - وهو معروف جيداً.,0
2,أسوأ فندق أقمت فيه على الإطلاق \r\nيستغرق تسجيل الوصول حوالي 30 دقيقة، باهظ الثمن، غرف قديمة جداً وقذرة، سجادة قذرة جداً... فى المجمل تجربة سيئة للغاية.\r\nمثل هذا الفندق سيكون في آسيا، نجمة واحدة، ويمكن أن أذهب في مايو.\r\n30 دولار أمريكي.\r\nالسعر في فندق نوفوتيل حوالي 250 دولار أمريكي\r\n,-1
3,بدون روح كأنه فندق ثلاثة نجوم \r\nبدون إدارة احترافية، فإن هذا الفندق يبدو كمركب ضائع في البحر.\r\nالغرف موزعة بطريقة عشوائية والإفطار هو نقانق باردة سخيفة ونفس السلطة القديمة في كل يوم.\r\nكان مكيف الهواء يثير الضوضاء في الغرفة ومن المستحيل إيقافه.\r\nولكن مركز صحي رائع\r\n,0
4,فندق جميل مع سوء الإدارة والخدمات. \r\nمن الخارج بدا مبشرًا، لكن خيبة الأمل بدأت في وقت الغداء، جودة الطعام سيئة سيئة سيئة، والحساء كان مزريًا جدًا، والفواكه جودتها سيئة (البكيخ كان متعفنًا، واو!). اشتكى زملائي من عدم وجود مناشف في غرفهم في حوالي الساعة 8 مساءً، والبعض لم تكن أسرّتهم مرتبة حتى الساعة 5 مساءً. تناولنا عشاء مكسيكيًا ومرة أخرى كانت قائمة الطعام مزرية. لن أقيم هنا أبدًا، على الأقل أتحدث عن نفسي!,-1


Only include postive and negative reviews:

In [8]:
data = data[data.polarity.isin([-1, 1])]

Apply text preprocessing/cleaning:

In [9]:
data['cleaned_text'] = data.text.apply(clean_text)

Remove empty string reviews:

In [10]:
data = data[data.cleaned_text != ""]
data.head(3)

,text,polarity,cleaned_text
0,"المكان الذي يمكنك فيه مراجعة الذات والتفكر هو كوكروبيت، غانا.... \r\nثمة الكثير عند زيارة غانا. وعلى الرغم من الفقر الذي سوف تلاحظه على طريقك إلى بيغ ميلي باكيارد، وجدت أن الناس في غانا يملكون ثراء القلب حتى رغم العوز. بيغ ميلي باكيارد هو مكان يمكنني فيه مراجعة الذات والتفكر . التقيت أشخاصاً كانوا يعملون على مبادرات مختلفة. بيغ ميلي هو مكان يمكننا فيه تبادل الملاحظات والآراء. بيغ ميلي مليء بالسلام وهادئ للغاية . الطعام ممتاز جداً! الطهاة ممتازون! وبالإضافة إلى ذلك, لا بد من الإشارة إلى أن العمال الذين لا يكلون، ويحرصون على سعادة نزلائهم هم من يجعلون المكان مضيافاً. هناك خيبة أمل واحدة كبيرة!! رغم أن الفنون والأشغال اليدوية يمكن شراؤها في الموقع إلا أن الأسعار غالية!!! مؤخراً أزالت السلطات في كوكروبيت محلات التجار الواقعة خارج شاطئ بيغ ميلي. كانت منتجات التجار أرخص! لا يزال هناك محل تجاري ""متجر مايسترو للفن الإفريقي في كوكروبيت"" يعرض الفنون والأشغال اليدوية الفريدة والملابس واللوحات. اشتريت لوحات مذهلة لايمكنك أن تجدها في بيغ ميلي. عندما تزور بيغ ميلي من فضلك خذ لحظة للذهاب إلى الشاطئ وعلى يمينك ستجد ""متجر مايسترو للفن الإفريقي"" المذهل. لا تنخدع بأسعار التجار في بيغ ميلي باكيارد. استمتع بالموسيقى والناس! الرجاء أن تقضي وقتاً ممتعاً.",1,المكان الذي يمكنك فيه مراجعه الذات والتفكر هو كوكروبيت غانا ثمه الكثير عند زياره غانا وعلي الرغم من الفقر الذي سوف تلاحظه علي طريقك الي بيغ ميلي باكيارد وجدت ان الناس في غانا يملكون ثراء القلب حتي رغم العوز بيغ ميلي باكيارد هو مكان يمكنني فيه مراجعه الذات والتفكر التقيت اشخاصا كانوا يعملون علي مبادرات مختلفه بيغ ميلي هو مكان يمكننا فيه تبادل الملاحظات والاراء بيغ ميلي مليء بالسلام وهادئ للغايه الطعام ممتاز جدا الطهاه ممتازون وبالاضافه الي ذلك لا بد من الاشاره الي ان العمال الذين لا يكلون ويحرصون علي سعاده نزلائهم هم من يجعلون المكان مضيافا هناك خيبه امل واحده كبيره رغم ان الفنون والاشغال اليدويه يمكن شراؤها في الموقع الا ان الاسعار غاليه مؤخرا ازالت السلطات في كوكروبيت محلات التجار الواقعه خارج شاطئ بيغ ميلي كانت منتجات التجار ارخص لا يزال هناك محل تجاري متجر مايسترو للفن الافريقي في كوكروبيت يعرض الفنون والاشغال اليدويه الفريده والملابس واللوحات اشتريت لوحات مذهله لايمكنك ان تجدها في بيغ ميلي عندما تزور بيغ ميلي من فضلك خذ لحظه للذهاب الي الشاطئ وعلي يمينك ستجد متجر مايسترو للفن الافريقي المذهل لا تنخدع باسعار التجار في بيغ ميلي باكيارد استمتع بالموسيقي والناس الرجاء ان تقضي وقتا ممتعا
2,أسوأ فندق أقمت فيه على الإطلاق \r\nيستغرق تسجيل الوصول حوالي 30 دقيقة، باهظ الثمن، غرف قديمة جداً وقذرة، سجادة قذرة جداً... فى المجمل تجربة سيئة للغاية.\r\nمثل هذا الفندق سيكون في آسيا، نجمة واحدة، ويمكن أن أذهب في مايو.\r\n30 دولار أمريكي.\r\nالسعر في فندق نوفوتيل حوالي 250 دولار أمريكي\r\n,-1,اسوا فندق اقمت فيه علي الاطلاق يستغرق تسجيل الوصول حوالي دقيقه باهظ الثمن غرف قديمه جدا وقذره سجاده قذره جدا في المجمل تجربه سيئه للغايه مثل هذا الفندق سيكون في اسيا نجمه واحده ويمكن ان اذهب في مايو دولار امريكي السعر في فندق نوفوتيل حوالي دولار امريكي
4,فندق جميل مع سوء الإدارة والخدمات. \r\nمن الخارج بدا مبشرًا، لكن خيبة الأمل بدأت في وقت الغداء، جودة الطعام سيئة سيئة سيئة، والحساء كان مزريًا جدًا، والفواكه جودتها سيئة (البكيخ كان متعفنًا، واو!). اشتكى زملائي من عدم وجود مناشف في غرفهم في حوالي الساعة 8 مساءً، والبعض لم تكن أسرّتهم مرتبة حتى الساعة 5 مساءً. تناولنا عشاء مكسيكيًا ومرة أخرى كانت قائمة الطعام مزرية. لن أقيم هنا أبدًا، على الأقل أتحدث عن نفسي!,-1,فندق جميل مع سوء الاداره والخدمات من الخارج بدا مبشرا لكن خيبه الامل بدات في وقت الغداء جوده الطعام سيئه سيئه سيئه والحساء كان مزريا جدا والفواكه جودتها سيئه البكيخ كان متعفنا واو اشتكي زملائي من عدم وجود مناشف في غرفهم في حوالي الساعه مساء والبعض لم تكن اسرتهم مرتبه حتي الساعه مساء تناولنا عشاء مكسيكيا ومره اخري كانت قائمه الطعام مزريه لن اقيم هنا ابدا علي الاقل اتحدث عن نفسي


Balance input data (same size postive and negative reviews):

In [11]:
min_sample = data.groupby(['polarity']).count().text.min()
input_data = pd.concat([data[data.polarity == 1].head(min_sample), 
                        data[data.polarity == -1].head(min_sample)])
input_data.groupby(['polarity']).count()

,text,cleaned_text
polarity,,
-1,2645,2645
1,2645,2645


# Create Numpy Arrays

In [12]:
X = input_data.cleaned_text.values

In [13]:
Y = np.asarray(input_data.polarity.values).astype('float32')
Y = Y.clip(0, 1)

# Create Sequences

In [14]:
num_words = 10000
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(X)

X = tokenizer.texts_to_sequences(X)

vocab_size = len(tokenizer.word_index) + 1
print("vocab size:", vocab_size)

vocab size: 48682


In [15]:
maxlen = 300
X = pad_sequences(X, padding='post', maxlen=maxlen)

Store tokenizer:

In [16]:
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

# Split Data

In [19]:
X_train, X_val, label_train, label_val = train_test_split(X, Y, test_size=0.35,
                                                            random_state=seed)
X_val, X_test, label_val, label_test = train_test_split(X_val, label_val, test_size=0.5,
                                                            random_state=seed)

Size of train and test data:

In [20]:
print("Training:", len(X_train), len(label_train))
print("Validation:", len(X_val), len(label_val))
print("Testing: ", len(X_test), len(label_test))

Training: 3438 3438
Validation: 926 926
Testing:  926 926


# Create Model

Bidirectional LSTM model:

In [ ]:
embedding_dim = 100
dropout = 0.5
opt = 'adam'
clear_session()

model = Sequential()
model.add(layers.Embedding(input_dim=num_words, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.Bidirectional(layers.LSTM(100, dropout=dropout, 
                                           recurrent_dropout=dropout, 
                                           return_sequences=True)))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(dropout))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(dropout))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(dropout))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=opt, 
              loss='binary_crossentropy', 
              metrics=['accuracy'])
model.summary()

# Training

In [22]:
history = model.fit(X_train, label_train,
                    epochs=4,
                    verbose=True,
                    validation_data=(X_val, label_val),
                    batch_size=64)
loss, accuracy = model.evaluate(X_train, label_train, verbose=True)
print("Training Accuracy: {:.4f}".format(accuracy))
loss_val, accuracy_val = model.evaluate(X_val, label_val, verbose=True)
print("Validation Accuracy:  {:.4f}".format(accuracy_val))

Epoch 1/4
54/54 [==============================] - 135s 2s/step - loss: 0.6932 - accuracy: 0.5159 - val_loss: 0.6816 - val_accuracy: 0.7300
Epoch 2/4
54/54 [==============================] - 129s 2s/step - loss: 0.6097 - accuracy: 0.6604 - val_loss: 0.3336 - val_accuracy: 0.8985
Epoch 3/4
54/54 [==============================] - 130s 2s/step - loss: 0.2343 - accuracy: 0.9349 - val_loss: 0.2273 - val_accuracy: 0.9320
Epoch 4/4
108/108 [==============================] - 18s 164ms/step - loss: 0.0734 - accuracy: 0.9907
Training Accuracy: 0.9907
29/29 [==============================] - 5s 165ms/step - loss: 0.1932 - accuracy: 0.9320
Validation Accuracy:  0.9320


# Evaluation

Predict:

In [23]:
pred_blind = model.predict(X_test, verbose=True)

29/29 [==============================] - 5s 162ms/step


Compre results:

In [24]:
df_blind = pd.DataFrame({'REAL': label_test, 
                         'PRED': pred_blind.reshape(pred_blind.shape[0],)})
df_blind = df_blind.reset_index()[['REAL', 'PRED']]
df_blind.PRED = df_blind.PRED.round()
error_records = df_blind[df_blind.REAL != df_blind.PRED]

In [25]:
print("Number of misclassified reviews: {} out of {}".format(error_records.shape[0], df_blind.shape[0]))
print("Blind Test Accuracy:  {:.4f}".format(accuracy_score(df_blind.REAL, df_blind.PRED)))

Number of misclassified reviews: 73 out of 926
Blind Test Accuracy:  0.9212


Sample outputs:

In [ ]:
df_blind.sample(n=3)

,REAL,PRED,TEXT
224,0.0,0.0,مختلف جدا عن فتره ازدهاره قديم وبحاجه الي ترميم عام موقع ممتاز مبني قديم وبالي وفي حاجه الي التجديدات والتحديثات يدعي صاحب الفندق ان المكانه التاريخيه تمنع وجود مخارج طوارئ واو اضاءه في حالات الطوارئ ومخارج حريق ذات علامات في هذا المبني المكون من طوابق والذي ليس فيه مصعد بل سلم مركزي ضيق في القاعه المضيفه لطيفه جدا لكن ليس هناك امر اخر جيد الافطار هو نفسه كل يوم وليس فيه ابتكار نفس الاومليت الممل كل يوم او الحبوب الجافه ولا توجد غرفه جلوس الخ ابحث عن مكان اخر الميزه الوحيده في هذا المكان هو الموقع والسعر في حي صاخب
94,1.0,1.0,فندق جميل فندق جميل ورائع وموظفينه بشوشين والغرف نظيفه وقد ماتكلم ماوفي حقه والصراحه الخدمه اعلي من النجوم الممنوحه للفندق جميل ورائع زي ماوضحت في ما سبق الغرف نظيفه والموقع جميل جدا جدا جدا وصراحه استمتعنا في الفندق وايضا الموظفين ممتازين وبشوشين واخلاقهم جميله وروحهم مرحه صراحه انا واصدقائي حنرجع مره اخري الي نفس الفندق صراحه ماقدر اوصف اكثر من كذا لانه شهادتي مجروحه والغرف والحمامات نظيفه والخدمه رائعه
214,0.0,0.0,اسوا خدمه عملاء لم نتوقع الكثير من فندق رخيص كان ملائما للمناسبه الخاصه بنا ولكن مكتب الاستقبال كان فظيعا طريقه التعامل السيئه كانت ظاهره من خلال عمل مكالمات شخصيه بينما كنا نحاول القيام باجراءات تسجيل الدخول عندما حاولنا الحصول علي مساعده قيل لنا انه لا يوجد مدير او مساعد في المكان بعد الساعه مساء بدا فريق العمل مستاء تقريبا لاننا حصلنا علي الغرفه بتكلفه رخيصه من


# Store Model and Weights

In [ ]:
with open('model_acc{}.json'.format(round(accuracy_val, 4)), 'w') as f:
    f.write(model.to_json())
    f.close()

In [ ]:
model.save_weights('model_acc{}.h5'.format(round(accuracy_val, 4)))